In [1]:
import numpy as np

# quara
from quara.objects.matrix_basis import (
    get_normalized_pauli_basis,
)
from quara.objects.elemental_system import ElementalSystem
from quara.objects.composite_system import CompositeSystem
from quara.objects.composite_system_typical import generate_composite_system
from quara.objects.tester_typical import (
    generate_tester_states,
    generate_tester_povms,
)
from quara.objects.qoperation_typical import generate_qoperation
from quara.protocol.qtomography.standard.standard_qst import StandardQst
from quara.protocol.qtomography.standard.standard_povmt import StandardPovmt
from quara.protocol.qtomography.standard.standard_qpt import StandardQpt

from quara.protocol.qtomography.standard.linear_estimator import LinearEstimator
from quara.protocol.qtomography.standard.loss_minimization_estimator import (
    LossMinimizationEstimator,
)
from quara.loss_function.weighted_probability_based_squared_error import (
    WeightedProbabilityBasedSquaredError,
    WeightedProbabilityBasedSquaredErrorOption,
)
from quara.minimization_algorithm.projected_gradient_descent_backtracking import (
    ProjectedGradientDescentBacktracking,
    ProjectedGradientDescentBacktrackingOption,
)

In [2]:
# 1-qubit Quantum State Tomography

In [3]:
mode = "qubit"
num = 1
c_sys = generate_composite_system(mode=mode, num=num)
print(c_sys)

elemental_systems:
[0] 0 (system_id=4337459408)

dim: 2
basis:
(array([[0.70710678+0.j, 0.        +0.j],
       [0.        +0.j, 0.70710678+0.j]]), array([[0.        +0.j, 0.70710678+0.j],
       [0.70710678+0.j, 0.        +0.j]]), array([[0.+0.j        , 0.-0.70710678j],
       [0.+0.70710678j, 0.+0.j        ]]), array([[ 0.70710678+0.j,  0.        +0.j],
       [ 0.        +0.j, -0.70710678+0.j]]))


In [4]:
# Testers
names = ["x", "y", "z"]
testers = generate_tester_povms(c_sys=c_sys, names = names)
for i, tester in enumerate(testers):
    print("Tester ", i, ":\n", tester)

Tester  0 :
 Type:
Povm

Dim:
2

Number of outcomes:
2

Vecs:
[[ 0.70710678  0.70710678  0.          0.        ]
 [ 0.70710678 -0.70710678  0.          0.        ]]
Tester  1 :
 Type:
Povm

Dim:
2

Number of outcomes:
2

Vecs:
[[ 0.70710678  0.          0.70710678  0.        ]
 [ 0.70710678  0.         -0.70710678  0.        ]]
Tester  2 :
 Type:
Povm

Dim:
2

Number of outcomes:
2

Vecs:
[[ 0.70710678  0.          0.          0.70710678]
 [ 0.70710678  0.          0.         -0.70710678]]
/Users/takanori/Work/dev/quara/quara/utils/matrix_util.py:195: ComplexWarning: Casting complex values to real discards the imaginary part
  tmp_hs = tmp_hs.astype(np.float64)


In [5]:
# Quantum State Tomography
seed = 7896
qst = StandardQst(testers, on_para_eq_constraint=True, schedules="all", seed=seed)

In [6]:
mode = "state"
name = "a"
true = generate_qoperation(mode=mode, name=name, c_sys=c_sys)
print(true)

Type:
State

Dim:
2

Vec:
[0.70710678 0.5        0.5        0.        ]


In [7]:
prob_dists = qst.calc_prob_dists(true)
print(prob_dists)

[[0.85355339 0.14644661]
 [0.85355339 0.14644661]
 [0.5        0.5       ]]


In [8]:
num_data = 1000
empi_dists = qst.generate_empi_dists(state=true, num_sum=num_data)
for f in empi_dists:
    print(f)

(1000, array([0.864, 0.136]))
(1000, array([0.844, 0.156]))
(1000, array([0.49, 0.51]))


In [9]:
estimator = LinearEstimator()
result = estimator.calc_estimate(qtomography=qst, empi_dists=empi_dists, is_computation_time_required=True)
estimate = result.estimated_qoperation
print(estimate)
print("is estimate physical? : ", estimate.is_physical())
print("\nEigenvalues are: ", estimate.calc_eigenvalues())

Type:
State

Dim:
2

Vec:
[ 0.70710678  0.51477374  0.48648947 -0.01414214]
is estimate physical? :  False

Eigenvalues are:  [1.0009311329913524, -0.0009311329913525568]


In [10]:
estimator = LossMinimizationEstimator()
loss = WeightedProbabilityBasedSquaredError(3)
loss_option = WeightedProbabilityBasedSquaredErrorOption("identity")
algo = ProjectedGradientDescentBacktracking()

obj_start = (qst.generate_empty_estimation_obj_with_setting_info().generate_origin_obj())
var_start = obj_start.to_var()
algo_option = ProjectedGradientDescentBacktrackingOption(mode_stopping_criterion_gradient_descent="sum_absolute_difference_variable", num_history_stopping_criterion_gradient_descent=1)

result = estimator.calc_estimate(qtomography=qst, empi_dists=empi_dists, loss=loss, loss_option=loss_option, algo=algo, algo_option=algo_option, is_computation_time_required=True)
estimate = result.estimated_qoperation
print(estimate)
print("\nis estimate physical? : ", estimate.is_physical())
print("\nEigenvalues are: ", estimate.calc_eigenvalues())

Type:
State

Dim:
2

Vec:
[ 0.70710678  0.51381689  0.4855852  -0.01411585]

is estimate physical? :  False

Eigenvalues are:  [1.000000019575682, -1.9575682325134736e-08]


In [11]:
# POVM Tomography

# Composite System
mode = "qubit"
num = 1
c_sys = generate_composite_system(mode=mode, num=num)
#print(c_sys)

# Testers
names = ["x0", "y0", "z0", "z1"]
testers = generate_tester_states(c_sys=c_sys, names = names)
povmt = StandardPovmt(testers, num_outcomes=2, on_para_eq_constraint=True, schedules="all")

In [13]:
mode = "povm"
name = "z"
true = generate_qoperation(mode=mode, name=name, c_sys=c_sys)
print(true)

Type:
Povm

Dim:
2

Number of outcomes:
2

Vecs:
[[ 0.70710678  0.          0.          0.70710678]
 [ 0.70710678  0.          0.         -0.70710678]]


In [14]:
prob_dists = povmt.calc_prob_dists(true)
print(prob_dists)

[[5.00000000e-01 5.00000000e-01]
 [5.00000000e-01 5.00000000e-01]
 [1.00000000e+00 2.22044605e-16]
 [0.00000000e+00 1.00000000e+00]]


In [16]:
num_data = 1000
empi_dists = povmt.generate_empi_dists(povm=true, num_sum=num_data)
for f in empi_dists:
    print(f)

(1000, array([0.507, 0.493]))
(1000, array([0.513, 0.487]))
(1000, array([1., 0.]))
(1000, array([0., 1.]))


In [17]:
estimator = LinearEstimator()
result = estimator.calc_estimate(qtomography=povmt, empi_dists=empi_dists, is_computation_time_required=True)
estimate = result.estimated_qoperation
print(estimate)
print("\nis estimate physical? : ", estimate.is_physical())
print("\nEigenvalues are:", estimate.calc_eigenvalues())

Type:
Povm

Dim:
2

Number of outcomes:
2

Vecs:
[[ 0.70710678  0.00989949  0.01838478  0.70710678]
 [ 0.70710678 -0.00989949 -0.01838478 -0.70710678]]

is estimate physical? :  False

Eigenvalues are: [[1.0002179524967092, -0.00021795249670929], [1.0002179524967092, -0.00021795249670906802]]


In [18]:
estimator = LossMinimizationEstimator()
loss = WeightedProbabilityBasedSquaredError(4)
loss_option = WeightedProbabilityBasedSquaredErrorOption("identity")
algo = ProjectedGradientDescentBacktracking()  
algo_option = ProjectedGradientDescentBacktrackingOption()

result = estimator.calc_estimate(qtomography=povmt, empi_dists=empi_dists, loss=loss, loss_option=loss_option, algo=algo, algo_option=algo_option, is_computation_time_required=True)
estimate = result.estimated_qoperation
print(estimate)
print("\nis estimate physical? : ", estimate.is_physical())
print("\nEigenvalues are:", estimate.calc_eigenvalues())

Type:
Povm

Dim:
2

Number of outcomes:
2

Vecs:
[[ 0.7071068   0.00989086  0.01836879  0.70679897]
 [ 0.70710676 -0.00989086 -0.01836879 -0.70679897]]

is estimate physical? :  False

Eigenvalues are: [[1.000000031919274, 4.3284874382681915e-15], [0.9999999999999956, -3.1919273836550485e-08]]


In [19]:
# Quantum Process Tomography

# Composite System
mode = "qubit"
num = 1
c_sys = generate_composite_system(mode=mode, num=num)
#print(c_sys)

# Testers
names_states = ["x0", "y0", "z0", "z1"]
testers_states = generate_tester_states(c_sys=c_sys, names = names_states)
names_povms = ["x", "y", "z"]
testers_povms = generate_tester_povms(c_sys=c_sys, names=names_povms)
qpt = StandardQpt(states=testers_states, povms=testers_povms, on_para_eq_constraint=True, schedules="all")

In [20]:
mode = "gate"
name = "hadamard"
true = generate_qoperation(mode=mode, name=name, c_sys=c_sys)
print(true)

Type:
Gate

Dim:
2

HS:
[[ 1.  0.  0.  0.]
 [ 0.  0.  0.  1.]
 [ 0.  0. -1.  0.]
 [ 0.  1.  0.  0.]]


In [21]:
prob_dists = qpt.calc_prob_dists(true)
print(prob_dists)

[[0.5 0.5]
 [0.5 0.5]
 [1.  0. ]
 [0.5 0.5]
 [0.  1. ]
 [0.5 0.5]
 [1.  0. ]
 [0.5 0.5]
 [0.5 0.5]
 [0.  1. ]
 [0.5 0.5]
 [0.5 0.5]]


In [22]:
num_data = 1000
empi_dists = qpt.generate_empi_dists(gate=true, num_sum=num_data)
for f in empi_dists:
    print(f)

(1000, array([0.497, 0.503]))
(1000, array([0.471, 0.529]))
(1000, array([1., 0.]))
(1000, array([0.505, 0.495]))
(1000, array([0., 1.]))
(1000, array([0.511, 0.489]))
(1000, array([1., 0.]))
(1000, array([0.524, 0.476]))
(1000, array([0.509, 0.491]))
(1000, array([0., 1.]))
(1000, array([0.488, 0.512]))
(1000, array([0.523, 0.477]))


In [23]:
estimator = LinearEstimator()
result = estimator.calc_estimate(qtomography=qpt, empi_dists=empi_dists, is_computation_time_required=True)
estimate = result.estimated_qoperation
print(estimate)
print("\nis estimate physical? : ", estimate.is_physical())
evals, evecs = np.linalg.eigh(estimate.to_choi_matrix())
print("\nEigenvalues are:", evals)

Type:
Gate

Dim:
2

HS:
[[ 1.     0.     0.     0.   ]
 [ 0.    -0.006  0.01   1.   ]
 [ 0.012 -0.07  -1.012  0.036]
 [ 0.032  0.968 -0.01  -0.014]]

is estimate physical? :  False

Eigenvalues are: [-0.03795514 -0.00386682  0.05060433  1.99121763]


In [24]:
estimator = LossMinimizationEstimator()
loss = WeightedProbabilityBasedSquaredError(12)
loss_option = WeightedProbabilityBasedSquaredErrorOption("identity")
algo = ProjectedGradientDescentBacktracking()  
algo_option = ProjectedGradientDescentBacktrackingOption()

result = estimator.calc_estimate(qtomography=qpt, empi_dists=empi_dists, loss=loss, loss_option=loss_option, algo=algo, algo_option=algo_option, is_computation_time_required=True)
estimate = result.estimated_qoperation
print(estimate)
print("\nis estimate physical? : ", estimate.is_physical())
evals, evecs = np.linalg.eigh(estimate.to_choi_matrix())
print("\nEigenvalues are:", evals)

Type:
Gate

Dim:
2

HS:
[[ 1.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 7.90187190e-04  6.20035750e-03  1.76091565e-02  9.84425049e-01]
 [ 1.51585771e-03 -3.51660812e-02 -9.79638897e-01  3.08360381e-02]
 [ 3.14696441e-02  9.65098548e-01 -3.30883759e-02 -7.06118340e-03]]

is estimate physical? :  False

Eigenvalues are: [-3.04846011e-08 -1.89456607e-08  3.43722091e-02  1.96562784e+00]
